<a href="https://colab.research.google.com/github/martin0310/Data-Mining/blob/main/Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd  
import io  
import requests  
  

In [ ]:
# add submit.csv
url = "http://140.123.173.10/truefake_dataset/submit.csv"  
s = requests.get(url).content  
submit_csv = pd.read_csv(io.StringIO(s.decode('utf-8')))
submit_csv.shape[0]

In [ ]:
# add train.csv
# label: 1 --> unreliable, 0 --> reliable
url = "http://140.123.173.10/truefake_dataset/train.csv"  
s = requests.get(url).content  
# train_csv = pd.read_csv(io.StringIO(s.decode('utf-8')))
# train 60%
# train_test 40%
get_rows = 4532
train_csv = pd.read_csv(io.StringIO(s.decode('utf-8')), nrows=get_rows)
skip_list = list(range(1, 4533)) + list(range(4533 + (get_rows * 4) // 6, train_csv.shape[0]))
train_test_csv = pd.read_csv(io.StringIO(s.decode('utf-8')), skiprows=[1, 2], nrows=(get_rows * 4) // 6)
# print(train_csv)
# print(train_test_csv)


In [ ]:
pp = list(range(111))
print(pp)

In [ ]:
# <class 'pandas.core.series.Series'> to list --> 2D
def pcss2List2D(pcss, pcss_len):
  ret_list = []
  for i in range(pcss_len):
    ret_list.append([])
    tmp_list = eval(pcss[i])
    for j in range(len(tmp_list)):
      ret_list[i].append(tmp_list[j])
  return ret_list

In [ ]:
# # add test.csv
# url = "http://140.123.173.10/truefake_dataset/test.csv"  
# s = requests.get(url).content  
# test_csv = pd.read_csv(io.StringIO(s.decode('utf-8')))
# test_csv.head()

In [ ]:
# train_csv preprocess...
from gensim.models import Word2Vec
import nltk
from nltk.cluster import KMeansClusterer
import jieba
import numpy as np
from sklearn import cluster
from sklearn import metrics
from sklearn.manifold import TSNE

In [ ]:
# add stopword.txt
# url = "https://gist.githubusercontent.com/larsyencken/1440509/raw/53273c6c202b35ef00194d06751d8ef630e53df2/stopwords.txt"  
# s = requests.get(url).content  
# stopword_arr = s.decode('utf-8').split("\n")
# stopword_arr.pop()
# for i in range(6):
#   stopword_arr.pop(0)

# print(stopword_arr)


In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer() # Stemming
stop_words = list(set(stopwords.words('english'))) # Stopword
stop_words_len = len(stop_words)
print(stop_words)


In [ ]:
# function to remove stop words
def remove_stop_words(word_list, stop_words_list):
  for stop_words_element in stop_words_list:
    while stop_words_element in word_list:
      word_list.remove(stop_words_element)
  return word_list

In [ ]:
import math
import csv
import time
from progressbar import *
# remove train_csv's stopwords

train_csv_rows_count = train_csv.shape[0]

# import to train2.csv
# id, title vector, text vector, author

with open('/content/train2.csv', 'w', newline='') as train2_file:
  writer = csv.writer(train2_file)
  writer.writerow(["id", "title_vector", "text_vector", "author", "label"])

progress = ProgressBar()
for i in progress(range(train_csv_rows_count)):
  # title
  title_arr_tmp = []
  if not isinstance(train_csv['title'][i],type(float('nan'))):
    title_arr_tmp = train_csv['title'][i].lower().replace(',', '').replace('?', '').replace(':', '').replace('"', '').replace('!', '').split(' ')
  title_arr_tmp = remove_stop_words(title_arr_tmp, stop_words)

  # text
  text_arr_tmp = []
  if not isinstance(train_csv['text'][i],type(float('nan'))):
    text_arr_tmp = train_csv['text'][i].lower().replace(',', '').replace('?', '').replace(':', '').replace('"', '').replace('!', '').split(' ')
  text_arr_tmp = remove_stop_words(text_arr_tmp, stop_words)

  # author
  author_tmp = 'UNKNOWNAUTHOR'
  if not isinstance(train_csv['author'][i],type(float('nan'))):
    author_tmp = train_csv['author'][i]

  # label
  label_tmp = 1
  if not isinstance(train_csv['label'][i],type(float('nan'))):
    label_tmp = train_csv['label'][i]

  # append to file
  with open('/content/train2.csv', 'a', newline='') as train2_file:
    writer = csv.writer(train2_file)
    writer.writerow([i, title_arr_tmp, text_arr_tmp, author_tmp, label_tmp])

print("train2.csv ok")

train2_csv = pd.read_csv('train2.csv')
print(train2_csv.head())
print(train2_csv.isnull().sum())


In [ ]:
# calculate author's score in train_csv
author_name_score = {}
for i in range(train2_csv.shape[0]):
  author_name = str(train2_csv['author'][i])
  
  if author_name not in author_name_score.keys():
    author_name_score[author_name] = 0

  if train2_csv['label'][i] == 1:
    author_name_score[author_name] = author_name_score[author_name] - 0.0001
  elif train_csv['label'][i] == 0:
    author_name_score[author_name] = author_name_score[author_name] + 0.0001

print(author_name_score)

In [ ]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
train2_title_vector_list = pcss2List2D(train2_csv['title_vector'], train2_csv.shape[0])
train2_text_vector_list = pcss2List2D(train2_csv['text_vector'], train2_csv.shape[0])

In [ ]:
# train2 word2vec

train2_title_word2vec_size = 5
train2_text_word2vec_size = 20

train2_title_word2vec_min_count = 1
train2_text_word2vec_min_count = 1

# title
train2_title_word2vec = Word2Vec(train2_title_vector_list, size=train2_title_word2vec_size, min_count=train2_title_word2vec_min_count)
print("title:")
# print(len(train2_title_word2vec.wv.vectors))
# print(train2_title_word2vec.wv.vocab.keys())
# text
train2_text_word2vec = Word2Vec(train2_text_vector_list, size=train2_text_word2vec_size, min_count=train2_text_word2vec_min_count)
print("text:")
# print(len(train2_text_word2vec.wv.vectors))
# print(train2_text_word2vec.wv.vocab.keys())

In [ ]:
print(len(train2_text_word2vec.wv.vectors))

In [ ]:
print(type(train2_text_word2vec.wv.vectors))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def identity_tokenizer(text):
  return text

# train2 tf-idf

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase = False)
# title
train2_title_result = tfidf.fit_transform(train2_title_vector_list)
# text
train2_text_result = tfidf.fit_transform(train2_text_vector_list)

# print(train2_title_result.todense())
# print(train2_text_result.todense())

In [ ]:
train2_title_tfidf_weighted = np.dot(train2_title_result.todense(), train2_title_word2vec.wv.vectors)
train2_text_tfidf_weighted = np.dot(train2_text_result.todense(), train2_text_word2vec.wv.vectors)
# print(train2_title_tfidf_weighted_list)
# print(train2_text_tfidf_weighted_list)

In [ ]:
# save to train3.csv
print("save to train3.csv")
with open('/content/train3.csv', 'w', newline='') as train3_file:
  writer = csv.writer(train3_file)
  col_list = []
  col_list.append("id")
  for i in range(train2_title_word2vec_size):
    col_list.append("title_vector_" + str(i))
  for i in range(train2_text_word2vec_size):
    col_list.append("text_vector_" + str(i))
  col_list.append("author")
  col_list.append("label")
  col_list.append("author_score")
  # print(len(col_list))
  writer.writerow(col_list)

In [ ]:
for i in range(train2_csv.shape[0]):
  col_list_value = []
  col_list_value.append(i)
  train2_title_tfidf_weighted_list = train2_title_tfidf_weighted[i].tolist()[0]
  # print(train2_title_tfidf_weighted_list)
  for j in range(len(train2_title_tfidf_weighted_list)):
    col_list_value.append(train2_title_tfidf_weighted_list[j])
  train2_text_tfidf_weighted_list = train2_text_tfidf_weighted[i].tolist()[0]
  for j in range(len(train2_text_tfidf_weighted_list)):
    col_list_value.append(train2_text_tfidf_weighted_list[j])
  col_list_value.append(train2_csv['author'][i])
  col_list_value.append(train2_csv['label'][i])
  col_list_value.append(author_name_score[train2_csv['author'][i]])

  with open('/content/train3.csv', 'a', newline='') as train3_file:
    writer = csv.writer(train3_file)
    writer.writerow(col_list_value)
    # print(len(col_list_value))

print("train3.csv ok")

In [ ]:
# save to train3_no_author_score.csv
print("save to train3_no_author_score.csv")
with open('/content/train3_no_author_score.csv', 'w', newline='') as train3_file:
  writer = csv.writer(train3_file)
  col_list = []
  col_list.append("id")
  for i in range(train2_title_word2vec_size):
    col_list.append("title_vector_" + str(i))
  for i in range(train2_text_word2vec_size):
    col_list.append("text_vector_" + str(i))
  col_list.append("author")
  col_list.append("label")
  # print(len(col_list))
  writer.writerow(col_list)
for i in range(train2_csv.shape[0]):
  col_list_value = []
  col_list_value.append(i)
  train2_title_tfidf_weighted_list = train2_title_tfidf_weighted[i].tolist()[0]
  # print(train2_title_tfidf_weighted_list)
  for j in range(len(train2_title_tfidf_weighted_list)):
    col_list_value.append(train2_title_tfidf_weighted_list[j])
  train2_text_tfidf_weighted_list = train2_text_tfidf_weighted[i].tolist()[0]
  for j in range(len(train2_text_tfidf_weighted_list)):
    col_list_value.append(train2_text_tfidf_weighted_list[j])
  col_list_value.append(train2_csv['author'][i])
  col_list_value.append(train2_csv['label'][i])

  with open('/content/train3_no_author_score.csv', 'a', newline='') as train3_file:
    writer = csv.writer(train3_file)
    writer.writerow(col_list_value)
    # print(len(col_list_value))

print("train3_no_author_score.csv ok")

In [ ]:
# --------------------------------------------------------
import math
import csv
import time
from progressbar import *
# remove train_csv's stopwords

train_test_csv_rows_count = train_test_csv.shape[0]

# import to train_test2.csv
# id, title vector, text vector, author

with open('/content/train_test2.csv', 'w', newline='') as train_test2_file:
  writer = csv.writer(train_test2_file)
  writer.writerow(["id", "title_vector", "text_vector", "author", "label"])

progress = ProgressBar()
for i in progress(range(train_test_csv_rows_count)):
  # title
  title_arr_tmp = []
  if not isinstance(train_test_csv['title'][i],type(float('nan'))):
    title_arr_tmp = train_test_csv['title'][i].lower().replace(',', '').replace('?', '').replace(':', '').replace('"', '').replace('!', '').split(' ')
  title_arr_tmp = remove_stop_words(title_arr_tmp, stop_words)

  # text
  text_arr_tmp = []
  if not isinstance(train_test_csv['text'][i],type(float('nan'))):
    text_arr_tmp = train_test_csv['text'][i].lower().replace(',', '').replace('?', '').replace(':', '').replace('"', '').replace('!', '').split(' ')
  text_arr_tmp = remove_stop_words(text_arr_tmp, stop_words)

  # author
  author_tmp = 'UNKNOWNAUTHOR'
  if not isinstance(train_test_csv['author'][i],type(float('nan'))):
    author_tmp = train_test_csv['author'][i]

  # label
  label_tmp = 1
  if not isinstance(train_test_csv['label'][i],type(float('nan'))):
    label_tmp = train_test_csv['label'][i]

  # append to file
  with open('/content/train_test2.csv', 'a', newline='') as train_test2_file:
    writer = csv.writer(train_test2_file)
    writer.writerow([i, title_arr_tmp, text_arr_tmp, author_tmp, label_tmp])

print("train_test2.csv ok")

train_test2_csv = pd.read_csv('train_test2.csv')
print(train_test2_csv.head())
print(train_test2_csv.isnull().sum())

In [ ]:
train_test2_title_vector_list = pcss2List2D(train_test2_csv['title_vector'], train_test2_csv.shape[0])
train_test2_text_vector_list = pcss2List2D(train_test2_csv['text_vector'], train_test2_csv.shape[0])

In [ ]:
# train_test2 word2vec

train_test2_title_word2vec_size = 5
train_test2_text_word2vec_size = 5

train_test2_title_word2vec_min_count = 1
train_test2_text_word2vec_min_count = 1

# title
train_test2_title_word2vec = Word2Vec(train_test2_title_vector_list, size=train_test2_title_word2vec_size, min_count=train_test2_title_word2vec_min_count)
print("title:")
# print(len(train_test2_title_word2vec.wv.vectors))
# print(train_test2_title_word2vec.wv.vocab.keys())
# text
train_test2_text_word2vec = Word2Vec(train_test2_text_vector_list, size=train_test2_text_word2vec_size, min_count=train_test2_text_word2vec_min_count)
print("text:")
# print(len(train_test2_text_word2vec.wv.vectors))
# print(train_test2_text_word2vec.wv.vocab.keys())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def identity_tokenizer(text):
  return text

# train2 tf-idf

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase = False)
# title
train_test2_title_result = tfidf.fit_transform(train_test2_title_vector_list)
# text
train_test2_text_result = tfidf.fit_transform(train_test2_text_vector_list)

# print(train_test2_title_result.todense())
# print(train_test2_text_result.todense())

In [ ]:
train_test2_title_tfidf_weighted = np.dot(train_test2_title_result.todense(), train_test2_title_word2vec.wv.vectors)
train_test2_text_tfidf_weighted = np.dot(train_test2_text_result.todense(), train_test2_text_word2vec.wv.vectors)

In [ ]:
# save to train_test3.csv
print("save to train_test3.csv")
with open('/content/train_test3.csv', 'w', newline='') as train_test3_file:
  writer = csv.writer(train_test3_file)
  col_list = []
  col_list.append("id")
  for i in range(train_test2_title_word2vec_size):
    col_list.append("title_vector_" + str(i))
  for i in range(train_test2_text_word2vec_size):
    col_list.append("text_vector_" + str(i))
  col_list.append("label")
  col_list.append("author_score")
  # print(len(col_list))
  writer.writerow(col_list)

In [ ]:
for i in range(train_test2_csv.shape[0]):
  col_list_value = []
  col_list_value.append(i)
  train_test2_title_tfidf_weighted_list = train_test2_title_tfidf_weighted[i].tolist()[0]
  # print(train_test2_title_tfidf_weighted_list)
  for j in range(len(train_test2_title_tfidf_weighted_list)):
    col_list_value.append(train_test2_title_tfidf_weighted_list[j])
  train_test2_text_tfidf_weighted_list = train_test2_text_tfidf_weighted[i].tolist()[0]
  for j in range(len(train_test2_text_tfidf_weighted_list)):
    col_list_value.append(train_test2_text_tfidf_weighted_list[j])

  col_list_value.append(train_test2_csv['label'][i])
  
  if train_test2_csv['author'][i] in author_name_score.keys():
    col_list_value.append(author_name_score[train_test2_csv['author'][i]])
  else:
    col_list_value.append(0)

  with open('/content/train_test3.csv', 'a', newline='') as train_test3_file:
    writer = csv.writer(train_test3_file)
    writer.writerow(col_list_value)
    # print(len(col_list_value))

print("train_test3.csv ok")

In [ ]:
from wordcloud import WordCloud
url = "http://140.123.173.10/truefake_dataset/train.csv"  
s = requests.get(url).content  
text = s.decode('utf-8')
print(type(text))
cloud = WordCloud(background_color='white').generate(text)
cloud.to_file('/content/output.png')

In [ ]:
# (string, filename)
def gen_wordcloud_to(s, f):
  cloud = WordCloud(background_color='white').generate(s)
  cloud.to_file(f)


In [ ]:
url = "http://140.123.173.10/truefake_dataset/train.csv"  
s = requests.get(url).content  
text = s.decode('utf-8')
gen_wordcloud_to(text, "/content/origin.png")
ttt1 = ' '.join(str(item) for innerlist in train_test2_title_vector_list for item in innerlist)
ttt2 = ' '.join(str(item) for innerlist in train_test2_text_vector_list for item in innerlist)
ttt = ttt1 + " " + ttt2
gen_wordcloud_to(ttt, "/content/no_stop_words.png")

In [ ]:
lll = ' '.join(str(item) for innerlist in train_test2_title_vector_list for item in innerlist)
print(lll)

In [ ]:
# # make doc tfidf matrix to dot with word2vec matrix

# # title
# train2_title_tf_matrix = []
# train2_title_idf_matrix = []
# train2_title_tfidf_matrix = []

# word_key_list = list(train2_title_word2vec.wv.vocab.keys())

# # title tf
# print("title tf")
# # row length: amount of documents
# # col length: amount of kwywords
# for i in range(train2_csv.shape[0]):
#   if i % 1000 == 0:
#     print(str(i / train2_csv.shape[0] * 100) + " %")
#   tf_list_tmp = []
#   for j in range(len(word_key_list)):
#     tf_counter_tmp = 0
#     for k in range(len(train2_title_vector_list[i])):
#       if train2_title_vector_list[i][k].__eq__(word_key_list[j]):
#         tf_counter_tmp = tf_counter_tmp + 1
#     # if tf_counter_tmp > 1:
#     #   print(word_key_list[j])
#     tf_list_tmp.append(tf_counter_tmp)
#   train2_title_tf_matrix.append(tf_list_tmp)
# print("train2_title_tf_matrix ok")
# # print(train2_title_tf_matrix)

# # title idf
# print("title idf")
# # row length: 1
# # col length: amount of kwywords
# for i in range(len(word_key_list)):
#   if i % 1000 == 0:
#     print(str(i / train2_csv.shape[0] * 100) + " %")
#   df_counter_tmp = 0
#   for j in range(train2_csv.shape[0]):
#     if word_key_list[i] in train2_title_vector_list[j]:
#       df_counter_tmp = df_counter_tmp + 1
#   idf_tmp = math.log(train2_csv.shape[0] / (1 + df_counter_tmp))
#   train2_title_idf_matrix.append(idf_tmp)
# print("train2_title_idf_matrix ok")
# # print(train2_title_idf_matrix)

# # title tfidf
# print("title tfidf")
# # row length: amount of documents
# # col length: amount of kwywords
# for i in range(len(train2_title_tf_matrix)):
#   tfidf_list_tmp = []
#   for j in range(len(train2_title_tf_matrix[i])):
#     tfidf_tmp = train2_title_tf_matrix[i][j] * train2_title_idf_matrix[i]
#     tfidf_list_tmp.append(tfidf_tmp)
#   train2_title_tfidf_matrix.append(tfidf_list_tmp)
# print("train2_title_tfidf_matrix ok")
# # print(train2_title_tfidf_matrix)


In [ ]:
# # text
# train2_text_tf_matrix = []
# train2_text_idf_matrix = []
# train2_text_tfidf_matrix = []

# word_key_list = list(train2_text_word2vec.wv.vocab.keys())

# # text tf
# print("text tf")
# # row length: amount of documents
# # col length: amount of kwywords
# for i in range(train2_csv.shape[0]):
#   if i % 2 == 0:
#     print(str(i / train2_csv.shape[0] * 100) + " %")
#   tf_list_tmp = []
#   for j in range(len(word_key_list)):
#     tf_counter_tmp = 0
#     for k in range(len(train2_text_vector_list[i])):
#       if train2_text_vector_list[i][k].__eq__(word_key_list[j]):
#         tf_counter_tmp = tf_counter_tmp + 1
#     # if tf_counter_tmp > 1:
#     #   print(word_key_list[j])
#     tf_list_tmp.append(tf_counter_tmp)
#   train2_text_tf_matrix.append(tf_list_tmp)
# print("train2_text_tf_matrix ok")
# # print(train2_text_tf_matrix)

# # text idf
# print("text idf")
# # row length: 1
# # col length: amount of kwywords
# for i in range(len(word_key_list)):
#   if i % 1000 == 0:
#     print(str(i / len(word_key_list) * 100) + " %")
#   df_counter_tmp = 0
#   for j in range(train2_csv.shape[0]):
#     if word_key_list[i] in train2_text_vector_list[j]:
#       df_counter_tmp = df_counter_tmp + 1
#   idf_tmp = math.log(train2_csv.shape[0] / (1 + df_counter_tmp))
#   train2_text_idf_matrix.append(idf_tmp)
# print("train2_text_idf_matrix ok")
# # print(train2_text_idf_matrix)

# # text tfidf
# print("text tfidf")
# # row length: amount of documents
# # col length: amount of kwywords
# for i in range(len(train2_text_tf_matrix)):
#   tfidf_list_tmp = []
#   for j in range(len(train2_text_tf_matrix[i])):
#     tfidf_tmp = train2_text_tf_matrix[i][j] * train2_text_idf_matrix[i]
#     tfidf_list_tmp.append(tfidf_tmp)
#   train2_text_tfidf_matrix.append(tfidf_list_tmp)
# print("train2_text_tfidf_matrix ok")
# # print(train2_text_tfidf_matrix)

In [ ]:
# def matrix_multiplier_2D(m1, m2):
#   res = [[0 for x in range(len(m2[0]))] for y in range(len(m1))]
#   # explicit for loops
#   for i in range(len(m1)):
#     for j in range(len(m2[0])):
#       for k in range(len(m2)):
#         # resulted matrix
#         res[i][j] += m1[i][k] * m2[k][j]
#   return res